<a href="https://colab.research.google.com/github/TonAraguahy/Machine-learning-Estoque-em-google-sheets/blob/main/Machine%20Learning%20para%20previs%C3%A3o%20de%20compra%20de%20estoque.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Machine learning estoque


In [ ]:
# Todas importações necessarias, como não tenho API google optei por auth para acessar a conta atraves da liberação.
from sklearn import tree
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
from google.colab import auth
auth.authenticate_user()
import pandas as pd
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)


# abre a planilha e seleciona a primeira aba, no outro caso não há aba especifica.
worksheet = gc.open('planilha1').worksheet('nome da aba')
# nesse teste, como as variaveis são poucas, se usar a mesma base existente com muitos dados para tentar prever o comportamento, ironicamente começou a dar numeros viciados, então preferi atuar com uma base menor deixando essa base em outra planilha e ir treinando-a conforme achasse necessário. Geralmente esse não é o melhor metodo para reforçar o treinamento, mas para esse caso foi funcional.
worksheet = gc.open('planilha2').worksheet('nome da aba')

# obter todos os valores da planilha, se houver dados demais e estiver usando o colab mesmo, talvez tenha que separar mais os modulos
rows = worksheet.get_all_values()
rows2 = worksheet2.get_all_values()

# remover a primeira linha (cabeçalho), se não houver é só tirar essa parte, não afetara o restante do codigo.
header = rows.pop(0)
header2 = rows2.pop(0)

#nesse caso "n" serviu, pois em row1 e row2 que estão abaixo, tanto o range, quando o "if row[n] são o mesmo numero, foi por acaso, mas acabou servindo. Dependendo da base, talvez tenha que tirar a variavel n e colocar os numeros correspondentes"
n = 5

#Usando laço for para fazer a ação em todas as colunas, e usando a função "del" que deleta a coluna no calculo, usei del row 4 vezes em "0" pois quando apaga a primeira coluna, ele já calcula como se a segunda coluna fosse a primeira... por isso apaguei quatro vezes, pois na minha base de dados as quatro primeiras colunas não são para calculo, são na verdade strings.
for row in rows:
    del row[0]
    del row[0]
    del row[0]
    del row[0]



# busca entre [] os valores contidos em rows2 que é a segunda planilha, busca apenas valores existentes, transforma todos em int, por isso é importante verificar se não existe alguma string para não dar erro
row1 = []
for row in rows2:
    if row[n]:
        row1.append([int(row[i]) for i in range(n)])

row2 = []
for row in rows2:
    if row[n]:
        row2.append(int(row[n]))




# criar lista com valores nulos na coluna 0, por isso o if está como if not row[0], pois nesse caso não existe a resposta, nesse caso especifico a resposta está na coluna A da planilha, perceba que em cima está na coluna da variavel "n" por isso foi necessário fazer o tratamento abaixo
row3 = []
for row in rows:
    if not row[0]:
        row3.append([int(row[i]) for i in range(1,6)]) 

# se estiver atuando com bases iguais, pode apenas ignorar essa etapa abaixo, porém no meu caso, eu preferi ter uma base menor de treinamento em que coloco manualmente os números que quero usar para treinar o comportamento da máquina, mas como as bases estão em ordens diferentes da primeira e segunda planilha, fiz o tratamento abaixo para que fiquem na mesma posição.

new_row3 = []
for row in row3:
  #Ou seja olhando para baixo troquei a posição do row[4] para primeira posição e por ai vai. Essa é a mesma posição de row1 e row2
    new_row = [row[4], row[0], row[1], row[2], row[3]]
    new_row3.append(new_row)

#como eu já havia feito todos os codigos antes com outra base e ideia como teste, preferi só tranformar a nova variavel em row3 para não precisar alterar os codigos de baixo
row3 = new_row3


print(row1)
print(row2)
print(row3)
x = row1
y = row2



ACURÁCIA DO PROJETO

In [ ]:
#import pandas as pd
#from sklearn.model_selection import train_test_split
#from sklearn import tree
#from sklearn.metrics import accuracy_score


# Dividindo os dados em conjuntos de treinamento e teste
#x_train, x_test, y_train, y_test = train_test_split(row1, row2, test_size=0.2)

# Treinando o classificador
#clf = tree.DecisionTreeRegressor()
#clf = clf.fit(x_train, y_train)

# Prevendo os resultados para os dados de teste
#predicted = clf.predict(x_test)

# Calculando a acurácia
#accuracy = accuracy_score(y_test, predicted)

# Imprimindo a acurácia
#print('Acurácia:', accuracy)



Treinar modelo e colocar dados na planilha

In [ ]:
#começa a treinar os dados e fazer a previsão dos dados vazios em row3
clf = tree.DecisionTreeRegressor()
clf = clf.fit(x, y)

test = row3

result= (clf.predict(test))

print(result)
import time

# Encontrar os índices dos espaços vazios na coluna, no inicio era coluna h
h_column = worksheet.col_values(5)
empty_indices = [i for i, value in enumerate(h_column) if not value]

# Atualizar as células vazias na coluna H com os valores de predicted_target, ou seja, começara a escrever na planilha o resultado
updates = []
for i, value in enumerate(result):
  if i in empty_indices:
    cell = worksheet.cell(i+2, 5)
    cell.value = value
    worksheet.update_cell(i+2, 5, value)
    #começou a dar muito erro pois o google coloca uma trava para que não faça x acessos por minuto, então precisei colocar esse temporizados de 0.1, talvez tenha que aumentar se o erro voltar no futuro
    time.sleep(0.1)

#por algum motivo o codigo acima ignora a linha 2, então coloquei manualmente para que result[0] que é o primeiro resultado, entre na linha 2 da coluna 5.
worksheet.update_cell(2, 5, result[0])

    
